In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np

In [ ]:
import keras
import os
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,MaxPool2D,Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19

In [ ]:
vgg_model = VGG19(input_shape=(224,224,3),weights='imagenet',include_top=False) #Tuning the vgg19 model and defining the input layer.
vgg_model.summary()

80134624/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# ignoring the exisiting model weights
for layer in vgg_model.layers:
  layer.trainable = False


In [ ]:
#Tuning the output layers.
x=Flatten()(vgg_model.output)
prediction = Dense(2,activation = 'softmax')(x)

In [ ]:
#Allocating the tuned model into an object using Model method
model = Model(inputs=vgg_model.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#Compiling the model
model.compile(optimizer='Adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
#Getting ready with the training data and testing data
trdata = ImageDataGenerator()
tr_data = trdata.flow_from_directory(directory='/content/drive/MyDrive/fire_detection/Fire_Dataset/Train',target_size=(224,224))

Found 2320 images belonging to 2 classes.


In [ ]:
tedata=ImageDataGenerator()
te_data = tedata.flow_from_directory(directory='/content/drive/MyDrive/fire_detection/Fire_Dataset/Test',target_size=(224,224))

Found 331 images belonging to 2 classes.


In [ ]:
tr_data.class_indices

{'Fire': 0, 'Neutral': 1}

In [ ]:
te_data.image_shape

(224, 224, 3)

In [ ]:
model.fit(tr_data,epochs=15, validation_data=te_data, validation_steps=6,verbose=1)

Epoch 1/15
73/73 [==============================] - 1703s 23s/step - loss: 2.0374 - accuracy: 0.9233 - val_loss: 2.2179 - val_accuracy: 0.9167
Epoch 2/15
73/73 [==============================] - 1694s 23s/step - loss: 0.5278 - accuracy: 0.9797 - val_loss: 3.8231 - val_accuracy: 0.9167
Epoch 3/15
73/73 [==============================] - 1692s 23s/step - loss: 0.1827 - accuracy: 0.9922 - val_loss: 4.2405 - val_accuracy: 0.9062
Epoch 4/15
73/73 [==============================] - 1680s 23s/step - loss: 0.0485 - accuracy: 0.9966 - val_loss: 3.8457 - val_accuracy: 0.9115
Epoch 5/15
73/73 [==============================] - 1685s 23s/step - loss: 0.0691 - accuracy: 0.9944 - val_loss: 5.6099 - val_accuracy: 0.9115
Epoch 6/15
73/73 [==============================] - 1676s 23s/step - loss: 0.0318 - accuracy: 0.9978 - val_loss: 5.6372 - val_accuracy: 0.9271
Epoch 7/15
73/73 [==============================] - 1672s 23s/step - loss: 0.0663 - accuracy: 0.9966 - val_loss: 6.3720 - val_accuracy: 0.8854

In [ ]:
model.save('/content/drive/MyDrive/fire_detection/Fire_Dataset/firemodel.h5')